In [1]:
import os
import pandas as pd
import requests
import tempfile
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
import giskard
from giskard.rag import KnowledgeBase, generate_testset
import nest_asyncio
nest_asyncio.apply()

c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Set up Ollama + embedding
api_base = "http://localhost:11434"
giskard.llm.set_llm_model("ollama/mistral", api_base="http://localhost:11434", disable_structured_output=True)
giskard.llm.set_embedding_model("ollama/nomic-embed-text", api_base=api_base)

In [4]:
# Load the Excel file with 'id' and 'link' columns
df_links = pd.read_excel("dataset.xlsx")
df_links.columns = df_links.columns.str.strip().str.lower()


In [5]:
# Use HuggingFace embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")

C:\Users\dunca\AppData\Local\Temp\ipykernel_17692\1947831796.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")


In [10]:
import traceback

for idx, row in df_links.iterrows():
    drug_id = row["id"]
    pdf_url = row["link"]

    print(f"\n📄 Processing {drug_id}...")

    try:
        # Download PDF
        response = requests.get(pdf_url)
        response.raise_for_status()

        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            temp_pdf_path = tmp_file.name

        # Load and split PDF into chunks
        loader = UnstructuredPDFLoader(temp_pdf_path)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
        documents = loader.load_and_split(text_splitter)

        if len(documents) < 2:
            print(f"[{drug_id}] ⚠️ Warning: only {len(documents)} chunk(s) extracted — may cause clustering issues.")

        # Embed documents
        vectorstore = DocArrayInMemorySearch.from_documents(documents, embedding=embedding_model)

        # Create knowledge base
        df_docs = pd.DataFrame([d.page_content for d in documents], columns=["text"])
        knowledge_base = KnowledgeBase(df_docs)

        # Generate 3 test questions
        testset = generate_testset(
            knowledge_base,
            num_questions=3,
            agent_description="A chatbot answering questions about medicine drugs"
        )

        # Save testset
        os.makedirs("testsets", exist_ok=True)
        out_path = f"testsets/testset_{drug_id}.jsonl"
        testset.save(out_path)
        print(f"[✓] Saved testset for {drug_id} to {out_path}")

    except Exception as e:
        print(f"[✗] Failed for {drug_id}: {e}")
        traceback.print_exc()



📄 Processing 1...


c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\venv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


2025-05-16 14:06:16,478 pid:22696 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\venv\lib\site-packages\umap\umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2025-05-16 14:07:45,028 pid:22696 MainThread giskard.rag  INFO     Found 4 topics in the knowledge base.


...

[✓] Saved testset for 25 to testsets/testset_25.jsonl

📄 Processing 26...
2025-05-16 15:29:44,549 pid:22696 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2025-05-16 15:29:49,873 pid:22696 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\venv\lib\site-packages\umap\umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
Generating questions: 100%|██████████| 3/3 [01:48<00:00, 36.13s/it]


[✓] Saved testset for 26 to testsets/testset_26.jsonl

📄 Processing 27...
2025-05-16 15:31:42,033 pid:22696 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2025-05-16 15:31:47,395 pid:22696 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\venv\lib\site-packages\umap\umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
Generating questions: 100%|██████████| 3/3 [01:43<00:00, 34.45s/it]

[✓] Saved testset for 27 to testsets/testset_27.jsonl


In [11]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [12]:
from langchain_ollama import OllamaLLM,OllamaEmbeddings 
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
import os
from dotenv import load_dotenv

load_dotenv()
MODEL = "llama2"
model = OllamaLLM(model=MODEL)

chain = (
    {
        "context": itemgetter("question") | vectorstore.as_retriever(),
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})  

In [17]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

CorrectnessMetric evaluation: 100%|██████████| 3/3 [00:41<00:00, 13.92s/it]


In [18]:
display(report)

Loading BokehJS ...

In [19]:
report.correctness_by_question_type()

,correctness
question_type,
complex,1.0
distracting element,1.0
simple,0.0


In [20]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
aaa3bd93-8110-47e9-b186-b2db1ea2434d,For which patient group is Disulfiram not reco...,"Disulfiram is contraindicated when used alone,...",Document 2: Disulfiram is absorbed slowly from...,[],"{'question_type': 'simple', 'seed_document_id'...","Based on the context provided, Disulfiram is n...",False,The agent provided an answer based on a specif...
